In [22]:
import pandas as pd
import numpy as np
import glob
import plotly_express as px
import os
os.chdir('C:\Finance_projects\Optiver-Realized-Volatility-Prediction\data')


In [23]:
unique_stocks_id = [  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  26,  27,  28,
        29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  46,  47,  48,  50,  51,  52,  53,  55,  56,  58,
        59,  60,  61,  62,  63,  64,  66,  67,  68,  69,  70,  72,  73,
        74,  75,  76,  77,  78,  80,  81,  82,  83,  84,  85,  86,  87,
        88,  89,  90,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       118, 119, 120, 122, 123, 124, 125, 126]

### plot scatter plot and correlation between first 10 min vol and 2nd 10 min vol.

In [24]:
## give a dataframe with two columns to plot scatter and return correlation coefficient, x axis is column 1 and y axis is column 2
def plot_scatter_n_correlation(df,st_id,method='pearson'):
    corr = df.corr(method=method)
    corr_coef = corr[corr.columns[0]][1] # convert from matrix to single number
    fig = px.scatter(df,x=df.columns[0],y=df.columns[1])
    fig.update_traces(textposition='top center')

    fig.update_layout(
        height=400,width=700,
        title_text='scatter plot, stock id: '+str(st_id)+ '. Correlation coef: '+ str(corr_coef) + ' method: '+method,
    )
    #fig.show()  # enable/disable plot

    return corr_coef

In [25]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

In [26]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))


In [27]:
corr_method = 'spearman' # 'pearson' or 'spearman'

## plot scatter plot and correlation between first 10 min vol and 2nd 10 min vol.
subset_paths = glob.glob('20_min_volatility.parquet/stock_*')

first_10_min_vol_n_target_corr_coef = {}

for path in subset_paths:
    st_id = int(path.split('\\')[1].split('_')[1].split('_')[0])
    if st_id != -1: # select stock id here
        vol_20_min = pd.read_parquet(path)
        #corr_coef = plot_20_min_vol_scatter(vol_20_min,st_id)
        first_10_min_vol_n_target_vol_df = pd.DataFrame({'first_10_min_vol':vol_20_min['first_10_min_vol'].values , 'target':vol_20_min['target'].values})
        corr_coef = plot_scatter_n_correlation(first_10_min_vol_n_target_vol_df,st_id,method=corr_method)
        first_10_min_vol_n_target_corr_coef[st_id] = corr_coef



In [28]:
## plot all stocks correlation coefficiceints together
coef_df=pd.DataFrame({'stock_id':first_10_min_vol_n_target_corr_coef.keys(),'correlation_coef':first_10_min_vol_n_target_corr_coef.values()})
px.bar(coef_df, x='stock_id',y='correlation_coef',title="Correlation coef. between " + str(first_10_min_vol_n_target_vol_df.columns[0]) + " and " + str(first_10_min_vol_n_target_vol_df.columns[1])+ " for all stocks"+ ". method: "+corr_method)

###  correlation between target vs. trade execution price at the available times. 

In [29]:
## correlation between target vs. trade execution price at the available times. 

train_target = pd.read_csv('train.csv')

# standard deviation (std) trade execution price vs. target
# and mean trade execution price vs. target

subset_paths = glob.glob('trade_train.parquet/stock_id=*')

std_price_n_target_vol_corr_coef = {}
mean_price_n_target_vol_corr_coef = {}
realized_vol_price_n_target_vol_corr_coef = {}

for path in subset_paths:
    st_id = int(path.split('\\')[1].split('_')[1].split('=')[1])
    # stock ids in [103,18,31,37] have DIFFERENT length of time_id compared to target so we exclude them for now
    if st_id != -1:#not in [103,18,31,37,110]: # select stock id here
        trade_train_st = pd.read_parquet(path)
        ## Std
        std_price = trade_train_st.groupby(by='time_id')['price'].std()
        std_price = pd.DataFrame(std_price).rename(columns={"price":"std_price"})
        target_st =train_target[train_target['stock_id']==st_id] 
        target_st.index = [target_st["time_id"]] 
        common_time_id = np.intersect1d(std_price.index.values, target_st['time_id'].values) # ensure that only common time ids in trade_train and target are used.
        std_price_n_target_vol_df = pd.DataFrame({'std_price':std_price.loc[common_time_id,"std_price"].values , 'target_vol':target_st.loc[common_time_id,"target"].values})
        corr_coef = plot_scatter_n_correlation(std_price_n_target_vol_df,st_id,method=corr_method)
        std_price_n_target_vol_corr_coef[st_id] = corr_coef
        
        ## mean
        mean_price = trade_train_st.groupby(by='time_id')['price'].mean()
        mean_price = pd.DataFrame(mean_price).rename(columns={"price":"mean_price"})
        target_st =train_target[train_target['stock_id']==st_id] 
        target_st.index = [target_st["time_id"]] 
        common_time_id = np.intersect1d(mean_price.index.values, target_st['time_id'].values) # ensure that only common time ids in trade_train and target are used.
        mean_price_n_target_vol_df = pd.DataFrame({'mean_price':mean_price.loc[common_time_id,"mean_price"].values , 'target_vol':target_st.loc[common_time_id,"target"].values})
        corr_coef = plot_scatter_n_correlation(mean_price_n_target_vol_df,st_id,method=corr_method)
        mean_price_n_target_vol_corr_coef[st_id] = corr_coef

        ## compute realized volality in the first 10 mins using trade execution price (instead of WAP)
        trade_train_st['log_ret_price'] = trade_train_st.groupby(by='time_id')['price'].apply(log_return).values
        trade_train_st = trade_train_st[~trade_train_st['log_ret_price'].isnull()]
        df_realized_vol_per_stock =  pd.DataFrame(trade_train_st.groupby(['time_id'])['log_ret_price'].agg(realized_volatility)).rename(columns={'log_ret_price':'realized_vol_price'})
        target_st =train_target[train_target['stock_id']==st_id] # use when not called above
        target_st.index = [target_st["time_id"]] # use when not called above
        common_time_id = np.intersect1d(df_realized_vol_per_stock.index.values, target_st['time_id'].values) # ensure that only common time ids in trade_train and target are used.
        realized_vol_price_n_target_vol_df = pd.DataFrame({'realized_vol_price':df_realized_vol_per_stock.loc[common_time_id,"realized_vol_price"].values , 'target_vol':target_st.loc[common_time_id,"target"].values})
        corr_coef = plot_scatter_n_correlation(realized_vol_price_n_target_vol_df,st_id,method=corr_method)
        realized_vol_price_n_target_vol_corr_coef[st_id] = corr_coef



In [30]:
## plot all stocks correlation coefficiceints together
coef_df=pd.DataFrame({'stock_id':std_price_n_target_vol_corr_coef.keys(),'correlation_coef':std_price_n_target_vol_corr_coef.values()})
fig = px.bar(coef_df, x='stock_id',y='correlation_coef',title="Correlation coef. between " + str(std_price_n_target_vol_df.columns[0]) + " and " + str(std_price_n_target_vol_df.columns[1])+ " for all stocks"+ ". method: "+corr_method)
fig.show()

## plot all stocks correlation coefficiceints together
coef_df=pd.DataFrame({'stock_id':mean_price_n_target_vol_corr_coef.keys(),'correlation_coef':mean_price_n_target_vol_corr_coef.values()})
fig1 =px.bar(coef_df, x='stock_id',y='correlation_coef',title="Correlation coef. between " + str(mean_price_n_target_vol_df.columns[0]) + " and " + str(mean_price_n_target_vol_df.columns[1])+ " for all stocks"+ ". method: "+corr_method)
fig1.show()

## plot all stocks correlation coefficiceints together
coef_df=pd.DataFrame({'stock_id':realized_vol_price_n_target_vol_corr_coef.keys(),'correlation_coef':realized_vol_price_n_target_vol_corr_coef.values()})
fig1 =px.bar(coef_df, x='stock_id',y='correlation_coef',title="Correlation coef. between " + str(realized_vol_price_n_target_vol_df.columns[0]) + " and " + str(realized_vol_price_n_target_vol_df.columns[1])+ " for all stocks"+ ". method: "+corr_method)
fig1.show()

### correlation between target vs. size at the available times.

In [31]:
## correlation between target vs. trade size at the available times. 

train_target = pd.read_csv('train.csv')

# standard deviation (std) trade size vs. target
# and mean trade size vs. target

subset_paths = glob.glob('trade_train.parquet/stock_id=*')

std_size_n_target_vol_corr_coef = {}
mean_size_n_target_vol_corr_coef = {}

for path in subset_paths:
    st_id = int(path.split('\\')[1].split('_')[1].split('=')[1])
    # stock ids in [103,18,31,37] have DIFFERENT length of time_id compared to target so we exclude them for now
    if st_id != -1:#not in [103,18,31,37,110]: # select stock id here
        trade_train_st = pd.read_parquet(path)
        ## Std
        std_size = trade_train_st.groupby(by='time_id')['size'].std()
        std_size = pd.DataFrame(std_size).rename(columns={"size":"std_size"})
        target_st =train_target[train_target['stock_id']==st_id] 
        target_st.index = [target_st["time_id"]] 
        common_time_id = np.intersect1d(std_size.index.values, target_st['time_id'].values) # ensure that only common time ids in trade_train and target are used.
        std_size_n_target_vol_df = pd.DataFrame({'std_size':std_size.loc[common_time_id,"std_size"].values , 'target_vol':target_st.loc[common_time_id,"target"].values})
        corr_coef = plot_scatter_n_correlation(std_size_n_target_vol_df,st_id,method=corr_method)
        std_size_n_target_vol_corr_coef[st_id] = corr_coef
        
        ## mean
        mean_size = trade_train_st.groupby(by='time_id')['size'].mean()
        mean_size = pd.DataFrame(mean_size).rename(columns={"size":"mean_size"})
        target_st =train_target[train_target['stock_id']==st_id] 
        target_st.index = [target_st["time_id"]] 
        common_time_id = np.intersect1d(mean_size.index.values, target_st['time_id'].values) # ensure that only common time ids in trade_train and target are used.
        mean_size_n_target_vol_df = pd.DataFrame({'mean_size':mean_size.loc[common_time_id,"mean_size"].values , 'target_vol':target_st.loc[common_time_id,"target"].values})
        corr_coef = plot_scatter_n_correlation(mean_size_n_target_vol_df,st_id,method=corr_method)
        mean_size_n_target_vol_corr_coef[st_id] = corr_coef

In [32]:
## plot all stocks correlation coefficiceints together
coef_df=pd.DataFrame({'stock_id':std_size_n_target_vol_corr_coef.keys(),'correlation_coef':std_size_n_target_vol_corr_coef.values()})
fig = px.bar(coef_df, x='stock_id',y='correlation_coef',title="Correlation coef. between " + str(std_size_n_target_vol_df.columns[0]) + " and " + str(std_size_n_target_vol_df.columns[1])+ " for all stocks"+ ". method: "+corr_method)
fig.show()

## plot all stocks correlation coefficiceints together
coef_df=pd.DataFrame({'stock_id':mean_size_n_target_vol_corr_coef.keys(),'correlation_coef':mean_size_n_target_vol_corr_coef.values()})
fig1 =px.bar(coef_df, x='stock_id',y='correlation_coef',title="Correlation coef. between " + str(mean_size_n_target_vol_df.columns[0]) + " and " + str(mean_size_n_target_vol_df.columns[1])+ " for all stocks"+ ". method: "+corr_method)
fig1.show()

### correlation between target vs. order_count at the available times. 

In [33]:
## correlation between target vs. trade order_count at the available times. 

train_target = pd.read_csv('train.csv')

# standard deviation (std) trade order_count vs. target
# and mean trade order_count vs. target

subset_paths = glob.glob('trade_train.parquet/stock_id=*')

std_order_count_n_target_vol_corr_coef = {}
mean_order_count_n_target_vol_corr_coef = {}

for path in subset_paths:
    st_id = int(path.split('\\')[1].split('_')[1].split('=')[1])
    # stock ids in [103,18,31,37] have DIFFERENT length of time_id compared to target so we exclude them for now
    if st_id != -1:#not in [103,18,31,37,110]: # select stock id here
        trade_train_st = pd.read_parquet(path)
        ## Std
        std_order_count = trade_train_st.groupby(by='time_id')['order_count'].std()
        std_order_count = pd.DataFrame(std_order_count).rename(columns={"order_count":"std_order_count"})
        target_st =train_target[train_target['stock_id']==st_id] 
        target_st.index = [target_st["time_id"]] 
        common_time_id = np.intersect1d(std_order_count.index.values, target_st['time_id'].values) # ensure that only common time ids in trade_train and target are used.
        std_order_count_n_target_vol_df = pd.DataFrame({'std_order_count':std_order_count.loc[common_time_id,"std_order_count"].values , 'target_vol':target_st.loc[common_time_id,"target"].values})
        corr_coef = plot_scatter_n_correlation(std_order_count_n_target_vol_df,st_id,method=corr_method)
        std_order_count_n_target_vol_corr_coef[st_id] = corr_coef
        
        ## mean
        mean_order_count = trade_train_st.groupby(by='time_id')['order_count'].mean()
        mean_order_count = pd.DataFrame(mean_order_count).rename(columns={"order_count":"mean_order_count"})
        target_st =train_target[train_target['stock_id']==st_id] 
        target_st.index = [target_st["time_id"]] 
        common_time_id = np.intersect1d(mean_order_count.index.values, target_st['time_id'].values) # ensure that only common time ids in trade_train and target are used.
        mean_order_count_n_target_vol_df = pd.DataFrame({'mean_order_count':mean_order_count.loc[common_time_id,"mean_order_count"].values , 'target_vol':target_st.loc[common_time_id,"target"].values})
        corr_coef = plot_scatter_n_correlation(mean_order_count_n_target_vol_df,st_id,method=corr_method)
        mean_order_count_n_target_vol_corr_coef[st_id] = corr_coef

In [34]:
## plot all stocks correlation coefficiceints together
coef_df=pd.DataFrame({'stock_id':std_order_count_n_target_vol_corr_coef.keys(),'correlation_coef':std_order_count_n_target_vol_corr_coef.values()})
fig = px.bar(coef_df, x='stock_id',y='correlation_coef',title="Correlation coef. between " + str(std_order_count_n_target_vol_df.columns[0]) + " and " + str(std_order_count_n_target_vol_df.columns[1])+ " for all stocks"+ ". method: "+corr_method)
fig.show()

## plot all stocks correlation coefficiceints together
coef_df=pd.DataFrame({'stock_id':mean_order_count_n_target_vol_corr_coef.keys(),'correlation_coef':mean_order_count_n_target_vol_corr_coef.values()})
fig1 =px.bar(coef_df, x='stock_id',y='correlation_coef',title="Correlation coef. between " + str(mean_order_count_n_target_vol_df.columns[0]) + " and " + str(mean_order_count_n_target_vol_df.columns[1])+ " for all stocks"+ ". method: "+corr_method)
fig1.show()